🔹 Step C — Punto di partenza

Adesso sei messo così:

Hai un corpus di 8323 giocatori, ognuno con una descrizione testuale text_profile.

Hai costruito un indice TF-IDF (Step B) con:

tfidf_vectorizer.joblib

tfidf_matrix.joblib

index_metadata.csv

Hai già una funzione search_players(query, top_k) che prende la query “così com’è” e calcola la similarità coseno sui profili.

👉 Con Step C vogliamo aggiungere uno strato di intelligenza sulla query:

capire se l’utente cerca un ruolo (playmaker / guardia / ala / centro),

capire se parla di skill specifiche (tiratore da 3, rimbalzista offensivo, difensore, passatore…),

riscrivere/arricchire la query in modo da dare più peso a questi concetti chiave quando la trasformiamo in TF-IDF.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento indice esistente
# ============================

MODELS_DIR = "../data/models"
VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# ============================
# 2. Normalizzazione base query
# ============================

def normalize_text(text: str) -> str:
    """Lowercase + rimozione spazi extra."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ============================
# 3. Regole di parsing semantico
# ============================

# Skill principali: tiri, rimbalzi, difesa, playmaking
SKILL_PATTERNS = {
    "shooting_3": [
        "tiratore da 3", "ottime percentuali da 3", "buon tiratore da 3",
        "da 3 punti", "tripla", "3 punti"
    ],
    "ft": [
        "buon ft", "ottimo ft", "buon tiratore di liberi", "ottimo tiratore di liberi",
        "buon ft%", "buon ft", "tiri liberi", "liberi"
    ],
    "reb": [
        "rimbalzista", "forte rimbalzista", "buon rimbalzista", "rimbalzi",
        "dominante a rimbalzo"
    ],
    "reb_off": [
        "rimbalzista offensivo", "rimbalzo offensivo", "forte rimbalzista offensivo"
    ],
    "playmaking": [
        "playmaker", "buon passatore", "assist", "creatore di gioco",
        "regista", "fa girare la squadra"
    ],
    "defense": [
        "difensore", "forte difensore", "buon difensore",
        "difesa", "difensiva", "specialista difensivo"
    ],
    "scorer": [
        "realizzatore", "scorer", "tanti punti", "tanti punti a partita",
        "prima opzione offensiva"
    ]
}

# Espansione di testo che useremo per ogni concetto (parole che esistono nei profili)
SEMANTIC_EXPANSIONS = {

    "shooting_3": "ottimo tiratore da 3 punti tiro da 3 3P% buon tiratore da 3",
    "ft":         "buon tiratore di tiri liberi FT% tiri liberi eccellente tiratore di tiri liberi",
    "reb":        "forte rimbalzista rimbalzi buon rimbalzista",
    "reb_off":    "molto forte a rimbalzo offensivo rimbalzista offensivo rimbalzi offensivi",
    "playmaking": "ottimo playmaker e passatore buon passatore assist",
    "defense":    "difensore di alto livello buon difensore profilo difensivo interessante",
    "scorer":     "segna in media molti punti punti a partita realizzatore"
}

# Peso (quante volte ripetere l’espansione nel testo pesato)
SEMANTIC_WEIGHTS = {

    "shooting_3": 4,
    "ft":         3,
    "reb":        3,
    "reb_off":    4,
    "playmaking": 3,
    "defense":    3,
    "scorer":     3
}

# ============================
# 4. Parsing della query -> intenti
# ============================

def parse_query_intents(raw_query: str):
    """
    Dato l'input naturale dell'utente, rileva:
    - ruolo target (PG/SG/SF/PF/C)
    - set di skill richieste (shooting_3, reb_off, difesa, playmaking, ...)
    Restituisce un dict con:
      {
        "normalized_query": ...,
        "roles": [...],
        "skills": [...],
      }
    """
    q = normalize_text(raw_query)
    detected_roles = set()
    detected_skills = set()

    # Skill
    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    intents = {
        "normalized_query": q,
        "roles": sorted(list(detected_roles)),
        "skills": sorted(list(detected_skills))
    }
    return intents

# ============================
# 5. Costruzione query pesata
# ============================

def build_weighted_query(raw_query: str, intents: dict) -> str:
    """
    Parte dalla query normale (normalizzata)
    e aggiunge "boost" lessicali per ruoli e skill individuate,
    ripetuti secondo i SEMANTIC_WEIGHTS.

    Restituisce una stringa che sarà passata al TF-IDF vectorizer.
    """
    base = intents.get("normalized_query", normalize_text(raw_query))
    boosted_parts = [base]

    # Skill
    for skill in intents.get("skills", []):
        if skill in SEMANTIC_EXPANSIONS:
            exp = SEMANTIC_EXPANSIONS[skill]
            w = SEMANTIC_WEIGHTS.get(skill, 1)
            boosted_parts.append((" " + exp) * w)

    weighted_query = " ".join(boosted_parts)
    return weighted_query.strip()

# ============================
# 6. Motore di ricerca "intelligente"
# ============================

def search_players_scout(query: str, top_k: int = 10):
    """
    Versione avanzata della search:
    - fa parsing semantico della query
    - costruisce una query pesata
    - calcola similarità coseno contro i profili
    - restituisce risultati + info sugli intenti interpretati
    """
    if not query or not isinstance(query, str):
        raise ValueError("La query deve essere una stringa non vuota.")

    # 1) Parsing semantico
    intents = parse_query_intents(query)

    # 2) Costruzione query pesata
    weighted_query = build_weighted_query(query, intents)

    # 3) TF-IDF della query pesata
    query_vec = vectorizer.transform([weighted_query])

    # 4) Similarità coseno
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).ravel()
    top_idx = np.argsort(sim_scores)[::-1][:top_k]

    results = metadata_df.iloc[top_idx].copy()
    results["similarity"] = sim_scores[top_idx]

    # Aggiungo informazioni sugli intenti interpretati (utile per debug / spiegazioni)
    results.attrs["intents"] = intents
    results.attrs["weighted_query"] = weighted_query

    return results

# ============================
# 7. Test rapido
# ============================

test_query = "Cerco una guardia tiratrice con ottime percentuali da 3, buon FT%, forte rimbalzista offensivo e buon difensore."

results = search_players_scout(test_query, top_k=5)

print("Query originale:")
print(test_query)
print("\nIntenti interpretati:")
print(results.attrs["intents"])

print("\nQuery pesata effettivamente usata per il TF-IDF:")
print(results.attrs["weighted_query"][:400], "...")  # tronco per non stampare un papiro

pd.set_option("display.max_colwidth", 200)
cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "similarity", "text_profile"] if c in results.columns]
print("\nTOP 5 risultati:")
display(results[cols_to_show])


Con questo Step C hai fatto un salto di qualità:

Punto di partenza: ricerca “stupida” → query usata così com’è:
TF-IDF(query) vs TF-IDF(text_profile).

Adesso:

La query viene interpretata con un mini motore rule-based:

riconosce ruoli (PG/SG/SF/PF/C),

riconosce skill (tiratore da 3, rimbalzista offensivo, difensore, playmaker, scorer…).

La query viene riscritta/pesata aggiungendo termini forti che sono presenti nei profili:

es. “ottimo tiratore da 3 punti tiro da 3 3P%…”

es. “molto forte a rimbalzo offensivo…”

I concetti chiave diventano molto più influenti nella similarità coseno.

Quindi ora il tuo sistema non è più un semplice “search testuale”, ma un primo motore intelligente per talent scout che capisce (in modo rule-based) cosa intende l’utente e adatta la ricerca in quella direzione.

Nel prossimo step (D) potrai:

rifinire il ranking,

aggiungere una spiegazione del tipo:

“Questo giocatore è in top-1 perché: ruolo SG, ottimo tiratore da 3, buon rimbalzista offensivo e buon difensore.”